In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
!wget 'https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0'

--2023-08-04 05:12:46--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2023-08-04 05:12:46--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1b6f47a510f488961599473e86.dl.dropboxusercontent.com/zip_download_get/BkgOqwvmgEEuucr8O5zWf3NCoMVz2Ix6zgqUUDWJe_zSBDSlQOs1KHh5ic9oPq0ZuK6iozjjCTI_0Khe8cDvj0eVyP9LOO_uAIaPf9zOwdbV9A# [following]
--2023-08-04 05:12:46--  https://uc1b6f47a510f488961599473e86.dl.dropboxusercontent.com/zip_download_get/BkgOqwvmgEEuucr8O5zWf3NCoMVz2Ix6zgqUUDWJe_zSBDSlQOs1KHh5ic9oPq0ZuK6iozjjCTI_0Khe8cDvj0eVyP9L

In [ ]:
!unzip '/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0'

Archive:  /content/AAAw1_JIzpuVvwteJCma0xMla?dl=0
mapname:  conversion of  failed
 extracting: train.csv               

1 archive had fatal errors.


In [ ]:
df=pd.read_csv('/content/train.csv')

In [ ]:
df

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [ ]:
df['target'].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
sent_lens=[]
for i in range(len(df)):
    a=df.loc[i,'question_text']
    try:
        sent_lens.append(len(word_tokenize(a)))

    except Exception as e:
        continue

max(sent_lens)

412

In [ ]:
df.loc[1,'question_text']

'Do you have an adopted dog, how would you encourage people to adopt and not shop?'

In [ ]:
import numpy as np
np.quantile(sent_lens,0.99),np.quantile(sent_lens,0.95),np.quantile(sent_lens,0.9)

(45.0, 31.0, 25.0)

In [ ]:
np.quantile(sent_lens,1),np.quantile(sent_lens,0.999),np.quantile(sent_lens,0.9999)

(412, 56.0, 64.0)

In [ ]:
a=[]
for i in sent_lens:
    if i>100:
        a.append(i)
        print(i)
print('.No. of sentences having 100  words or more are',len(a) )

361
101
155
147
182
108
412
123
177
125
111
.No. of sentences having 100  words or more are 11


In [ ]:
import numpy as np
max_len=80


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Embedding,Dropout,Input

In [ ]:
from sklearn.model_selection import train_test_split
X=df['question_text']
y=df['target']
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
tok=Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(x_train)
seq=tok.texts_to_sequences(x_train)
x_train_seq=pad_sequences(seq,maxlen=max_len)

In [ ]:
seq1=tok.texts_to_sequences(x_test)
x_test_seq=pad_sequences(seq1,maxlen=max_len)

In [ ]:
x_train_seq.shape,x_test_seq.shape

((1044897, 80), (261225, 80))

In [ ]:
vocab_size=len(tok.word_index)
vocab_size

195852

In [ ]:
!wget 'http://nlp.stanford.edu/data/glove.42B.300d.zip'

--2023-08-04 09:46:31--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2023-08-04 09:46:31--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2023-08-04 09:46:31--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [ ]:
!unzip '/content/glove.42B.300d.zip.1'

Archive:  /content/glove.42B.300d.zip.1
  inflating: glove.42B.300d.txt      


In [ ]:
embeding_index={}

f=open('/content/glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    mtx=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=mtx
f.close()

In [ ]:
!rm '/content/glove.42B.300d.zip'

rm: cannot remove '/content/glove.42B.300d.zip': No such file or directory


In [ ]:
embedding_matrix=np.zeros((vocab_size+1,300))
not_available=0
for word,i in tok.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

    else:
        not_available+=1
not_available

61908

In [ ]:
embedding_matrix.shape,embedding_matrix

((195853, 300),
 array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
          0.21199   , -0.0094357 ],
        [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
          0.46514001,  0.11363   ],
        ...,
        [-0.2922    , -0.20874999,  0.17894   , ...,  0.39546999,
          0.10891   , -0.041327  ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]))

In [ ]:
model=Sequential()
model.add((Embedding(input_dim=(vocab_size+1),output_dim=300,input_length=max_len,
                          mask_zero=True,
                          weights=[embedding_matrix],trainable=False)))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(25,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 300)           58755900  
                                                                 
 lstm (LSTM)                 (None, 80, 64)            93440     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 25)                825       
                                                                 
 dropout (Dropout)           (None, 25)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                416       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0

In [ ]:
model.fit(x_train_seq,y_train,
          validation_data=[x_test_seq,y_test],epochs=6,batch_size=2000)

Epoch 1/6
523/523 [==============================] - 365s 670ms/step - loss: 0.1837 - accuracy: 0.9401 - val_loss: 0.1180 - val_accuracy: 0.9530
Epoch 2/6
523/523 [==============================] - 261s 499ms/step - loss: 0.1224 - accuracy: 0.9541 - val_loss: 0.1100 - val_accuracy: 0.9560
Epoch 3/6
523/523 [==============================] - 264s 505ms/step - loss: 0.1138 - accuracy: 0.9566 - val_loss: 0.1069 - val_accuracy: 0.9576
Epoch 4/6
523/523 [==============================] - 259s 496ms/step - loss: 0.1089 - accuracy: 0.9582 - val_loss: 0.1053 - val_accuracy: 0.9577
Epoch 5/6
523/523 [==============================] - 251s 479ms/step - loss: 0.1053 - accuracy: 0.9590 - val_loss: 0.1052 - val_accuracy: 0.9577
Epoch 6/6
523/523 [==============================] - 264s 505ms/step - loss: 0.1024 - accuracy: 0.9596 - val_loss: 0.1040 - val_accuracy: 0.9578


In [ ]:
model.fit(x_train_seq,y_train,
          validation_data=[x_test_seq,y_test],epochs=6,batch_size=20000)

Epoch 1/6
53/53 [==============================] - 45s 841ms/step - loss: 0.0978 - accuracy: 0.9611 - val_loss: 0.1052 - val_accuracy: 0.9589
Epoch 2/6
53/53 [==============================] - 44s 836ms/step - loss: 0.0961 - accuracy: 0.9616 - val_loss: 0.1059 - val_accuracy: 0.9587
Epoch 3/6
53/53 [==============================] - 44s 833ms/step - loss: 0.0958 - accuracy: 0.9619 - val_loss: 0.1053 - val_accuracy: 0.9584
Epoch 4/6
53/53 [==============================] - 44s 835ms/step - loss: 0.0951 - accuracy: 0.9619 - val_loss: 0.1055 - val_accuracy: 0.9583
Epoch 5/6
53/53 [==============================] - 44s 832ms/step - loss: 0.0946 - accuracy: 0.9620 - val_loss: 0.1055 - val_accuracy: 0.9583
Epoch 6/6
53/53 [==============================] - 44s 837ms/step - loss: 0.0940 - accuracy: 0.9623 - val_loss: 0.1057 - val_accuracy: 0.9585


In [ ]:
from sklearn import metrics as m
pred=model.predict(x_test_seq)
print('roc_auc_score',m.roc_auc_score(y_test,pred))

8164/8164 [==============================] - 279s 34ms/step
roc_auc_score 0.9639209209071631


In [ ]:
predicts=np.where(pred>0.5,1,0)
print('classification report\n',m.classification_report(y_test,predicts))
print('confusion_matrix\n',m.confusion_matrix(y_test,predicts))


classification report
               precision    recall  f1-score   support

           0       0.98      0.98      0.98    245042
           1       0.67      0.64      0.66     16183

    accuracy                           0.96    261225
   macro avg       0.82      0.81      0.82    261225
weighted avg       0.96      0.96      0.96    261225

confusion_matrix
 [[239990   5052]
 [  5780  10403]]


In [ ]:
!wget 'http://vectors.nlpl.eu/repository/20/0.zip'

--2023-08-04 08:27:35--  http://vectors.nlpl.eu/repository/20/0.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344050746 (328M) [application/zip]
Saving to: ‘0.zip’

0.zip               100%[===================>] 328.11M  18.4MB/s    in 18s     

2023-08-04 08:27:55 (17.8 MB/s) - ‘0.zip’ saved [344050746/344050746]



In [ ]:
!unzip '/content/0.zip'

Archive:  /content/0.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [ ]:
embedding_index1={}
f=open('/content/model.txt',encoding='utf-8')
for i,line in enumerate(f):
    if i==0:
        continue
    values=line.split()
    words=values[0].split('_')[0]
    matrix=np.asarray(values[1:],dtype='float64')
    embedding_index1[words]=matrix

f.close()
#print(embedding_index1)

In [ ]:
embedding_matrix1=np.zeros((vocab_size+1,300))
not_avail=0
for word,matrix in tok.word_index.items():
    em_vec=embedding_index1.get(word)
    if em_vec is not None:
        embedding_matrix1[matrix]=em_vec
    else:
        not_avail+=1

print(not_avail)


155564


In [ ]:
embedding_matrix1.shape,embedding_matrix.shape,not_available,not_avail

((196049, 300), (196049, 300), 62121, 155564)

In [ ]:
model1=Sequential()
model1.add((Embedding(input_dim=(vocab_size+1),output_dim=300,input_length=max_len,
                          mask_zero=True,
                          weights=[embedding_matrix1],trainable=False)))
model1.add(LSTM(64,return_sequences=True))
model1.add(LSTM(32))
model1.add(Dense(25,activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(16,activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(8,activation='relu'))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 80, 300)           58814700  
                                                                 
 lstm_6 (LSTM)               (None, 80, 64)            93440     
                                                                 
 lstm_7 (LSTM)               (None, 32)                12416     
                                                                 
 dense_10 (Dense)            (None, 25)                825       
                                                                 
 dropout_4 (Dropout)         (None, 25)                0         
                                                                 
 dense_11 (Dense)            (None, 16)                416       
                                                                 
 dropout_5 (Dropout)         (None, 16)               

In [ ]:
model1.fit(x_train_seq,y_train,
          validation_data=[x_test_seq,y_test],epochs=6,batch_size=2000)

Epoch 1/6
523/523 [==============================] - 271s 501ms/step - loss: 0.1574 - accuracy: 0.9439 - val_loss: 0.1201 - val_accuracy: 0.9521
Epoch 2/6
523/523 [==============================] - 253s 483ms/step - loss: 0.1221 - accuracy: 0.9518 - val_loss: 0.1113 - val_accuracy: 0.9558
Epoch 3/6
523/523 [==============================] - 252s 482ms/step - loss: 0.1155 - accuracy: 0.9539 - val_loss: 0.1110 - val_accuracy: 0.9574
Epoch 4/6
523/523 [==============================] - 248s 474ms/step - loss: 0.1108 - accuracy: 0.9554 - val_loss: 0.1115 - val_accuracy: 0.9577
Epoch 5/6
523/523 [==============================] - 245s 469ms/step - loss: 0.1067 - accuracy: 0.9571 - val_loss: 0.1086 - val_accuracy: 0.9584
Epoch 6/6
523/523 [==============================] - 244s 467ms/step - loss: 0.1033 - accuracy: 0.9583 - val_loss: 0.1052 - val_accuracy: 0.9582


In [ ]:
pred1=model1.predict(x_test_seq)
print('roc_auc_score',m.roc_auc_score(y_test,pred1))


8164/8164 [==============================] - 295s 36ms/step
roc_auc_score 0.9638871927831558


In [ ]:
predicts1=np.where(pred1>0.5,1,0)
print('classification report\n',m.classification_report(y_test,predicts1))
print('confusion_matrix\n',m.confusion_matrix(y_test,predicts1))

classification report
               precision    recall  f1-score   support

           0       0.98      0.98      0.98    244913
           1       0.67      0.65      0.66     16312

    accuracy                           0.96    261225
   macro avg       0.82      0.81      0.82    261225
weighted avg       0.96      0.96      0.96    261225

confusion_matrix
 [[239733   5180]
 [  5744  10568]]


In [ ]:
pred.shape

(261225, 1)

In [ ]:
final_model=model
pred=final_model.predict(x_test_seq)
#predictions=pd.DataFrame({'target':pred})



8164/8164 [==============================] - 315s 39ms/step


In [ ]:
pred.shape,pred.reshape(-1,1),pred

((261225, 1),
 array([[4.6802495e-09],
        [1.8024726e-05],
        [4.9786445e-01],
        ...,
        [4.6823923e-09],
        [1.0332413e-02],
        [1.5137845e-04]], dtype=float32),
 array([[4.6802495e-09],
        [1.8024726e-05],
        [4.9786445e-01],
        ...,
        [4.6823923e-09],
        [1.0332413e-02],
        [1.5137845e-04]], dtype=float32))

In [ ]:
predictions=pd.DataFrame({'target':pred.flatten()})

In [ ]:
predictions.to_csv('Pralay_kumar_das_spam_filter.csv',index=False)